<a href="https://colab.research.google.com/github/756y/colab/blob/main/%E5%9F%BA%E5%9B%A0%E6%BC%94%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 基因演算法

In [4]:
import random  # 引入隨機數模組，用於隨機生成數值
import statistics  # 引入統計模組，用於計算標準差

# ========================== 配置參數 ==========================
num_lights = 4  # 設定路口數量（即信號燈數量），這裡有四個路口
population_size = 40  # 種群大小（即候選解的數量），每代進化會有40個解
generations = 100  # 進化迭代次數，表示演化的世代數
mutation_rate = 0.2  # 突變率，表示在每次突變操作中變異的機率
green_time_range = (30, 90)  # 設定綠燈時間範圍，信號燈的綠燈時間會在30秒到90秒之間

# ========================== 車流量設置 ==========================
traffic_flow_peak = [random.randint(500, 800) for _ in range(num_lights)]  # 高峰期的車流量隨機生成，範圍在500到800之間
traffic_flow_offpeak = [random.randint(150, 330) for _ in range(num_lights)]  # 非高峰期的車流量隨機生成，範圍在150到330之間

# ========================== 基本限制條件 ==========================
pedestrian_min_time = 20  # 行人綠燈的最小時間限制，確保行人過街有足夠的時間
min_vehicle_throughput = 200  # 每單位綠燈時間最小車輛通過量，這是基於流量和綠燈時間的車輛通行速率

max_offpeak_green_time = 60  # 非高峰期最大綠燈時間限制，確保非高峰期間的綠燈時間不過長
min_peak_green_time = 60  # 設定高峰期的最小綠燈時間，保證高峰期有足夠的時間處理車流量

emergency_vehicle_priority = [0, 1]  # 應急車輛優先路口，假設0號和1號路口有應急車輛優先權

# ========================== 函數定義 ==========================
# 計算等待時間，根據信號燈綠燈時間與車流量進行計算
def calculate_wait_time(signal_times, traffic_flow):
    total_wait_time = 0  # 總等待時間初始化
    for i in range(num_lights):  # 遍歷每個路口
        green_time = signal_times[i]  # 獲取當前路口的綠燈時間
        flow = traffic_flow[i]  # 獲取當前路口的車流量
        cycle_time = sum(signal_times)  # 信號周期時間（綠燈加紅燈時間總和）
        red_time = cycle_time - green_time  # 紅燈時間等於周期時間減去綠燈時間

        # 計算有效車流量，考慮每單位綠燈時間可以處理的最小車輛數
        effective_flow = min(flow, green_time * min_vehicle_throughput)
        # 計算等待時間，這個是根據紅燈時間與有效流量的比例來估算的
        wait_time = (red_time / cycle_time) * effective_flow
        total_wait_time += wait_time  # 累加總等待時間
    return total_wait_time

# 計算適應度，這是目標函數，我們希望最大化適應度，實際上是最小化總等待時間
def fitness(signal_times, traffic_flow, is_offpeak=False):
    wait_time = calculate_wait_time(signal_times, traffic_flow)  # 計算總等待時間

    # 懲罰行人綠燈時間過短（確保行人有足夠的過街時間）
    penalty_pedestrian = sum(max(pedestrian_min_time - t, 0) for t in signal_times)

    # 懲罰非高峰期綠燈時間過長的情況
    penalty_offpeak = 0
    if is_offpeak:
        penalty_offpeak = sum(max(t - max_offpeak_green_time, 0) for t in signal_times)

    # 懲罰高峰期綠燈時間過短
    penalty_peak = 0
    if not is_offpeak:  # 如果是高峰期
        penalty_peak = sum(max(min_peak_green_time - t, 0) for t in signal_times)

    # 計算綠燈時間的標準差，標準差越大表示信號燈時間越不均衡，我們希望它較小
    std_dev = statistics.stdev(signal_times) if len(signal_times) > 1 else 0
    penalty_balance = std_dev * 2  # 根據標準差進行懲罰

    # 返回負的適應度值，因為我們要最小化總等待時間
    return -(wait_time + penalty_pedestrian * 10 + penalty_offpeak * 10 + penalty_peak * 10 + penalty_balance)


# 初始化種群，每個個體是一個路口的綠燈時間配置
def initialize_population():
    return [
        [random.randint(green_time_range[0], green_time_range[1]) for _ in range(num_lights)]
        for _ in range(population_size)
    ]

# 突變操作，根據突變率隨機改變某個路口的綠燈時間
def mutate(signal_times):
    if random.random() < mutation_rate:  # 隨機決定是否進行突變
        idx = random.randint(0, num_lights - 1)  # 隨機選擇一個路口
        signal_times[idx] = random.randint(green_time_range[0], green_time_range[1])  # 隨機設置綠燈時間
    return signal_times

# 交叉操作，選擇兩個父代的綠燈時間配置並進行交叉產生子代
def crossover(parent1, parent2):
    point = random.randint(1, num_lights - 1)  # 隨機選擇交叉點
    return parent1[:point] + parent2[point:]  # 交叉兩個父代生成一個子代

# 選擇操作，根據適應度選擇父代，適應度越高的個體被選中的機率越大
def selection(population, traffic_flow, is_offpeak=False):
    fitness_values = [fitness(ind, traffic_flow, is_offpeak) for ind in population]  # 計算所有個體的適應度
    total_fitness = sum(fitness_values)  # 所有個體的適應度總和
    probabilities = [f / total_fitness for f in fitness_values]  # 根據適應度計算選擇概率
    return population[random.choices(range(len(population)), probabilities, k=1)[0]]  # 根據概率選擇父代

# 優先處理應急車輛，給應急車輛路口分配更多的綠燈時間
def prioritize_emergency(signal_times):
    for i in emergency_vehicle_priority:  # 優先給應急車輛路口分配綠燈時間
        signal_times[i] = max(signal_times[i], int(1.5 * pedestrian_min_time))  # 保證應急車輛綠燈時間足夠
    return signal_times

# ========================== 主程序 ==========================
# 優化交通流量配置
def optimize_traffic(traffic_flow, is_offpeak=False):
    population = initialize_population()  # 初始化種群
    for generation in range(generations):  # 進行指定代數的進化
        population.sort(key=lambda x: fitness(x, traffic_flow, is_offpeak), reverse=True)  # 按適應度排序
        new_population = population[:population_size // 2]  # 保留適應度最高的一半個體
        while len(new_population) < population_size:  # 填充新種群
            parent1 = selection(population, traffic_flow, is_offpeak)  # 選擇父代1
            parent2 = selection(population, traffic_flow, is_offpeak)  # 選擇父代2
            child = mutate(crossover(parent1, parent2))  # 交叉產生子代並進行突變
            child = prioritize_emergency(child)  # 優先處理應急車輛
            new_population.append(child)  # 添加子代到新種群
        population = new_population  # 更新種群
    return max(population, key=lambda x: fitness(x, traffic_flow, is_offpeak))  # 返回適應度最好的個體

# ========================== 結果輸出 ==========================
best_signal_peak = optimize_traffic(traffic_flow_peak)  # 優化高峰期信號燈配置
best_signal_offpeak = optimize_traffic(traffic_flow_offpeak, is_offpeak=True)  # 優化非高峰期信號燈配置

wait_time_peak = calculate_wait_time(best_signal_peak, traffic_flow_peak)  # 計算高峰期總等待時間
wait_time_offpeak = calculate_wait_time(best_signal_offpeak, traffic_flow_offpeak)  # 計算非高峰期總等待時間

# 輸出高峰期和非高峰期的結果
print("========== 高峰期結果 ==========")
print("最佳信號燈綠燈時間配置：", best_signal_peak)
print("高峰期總等待時間：", wait_time_peak)
print("適應度：", -fitness(best_signal_peak, traffic_flow_peak))  # 輸出高峰期適應度

print("\n========== 非高峰期結果 ==========")
print("最佳信號燈綠燈時間配置：", best_signal_offpeak)
print("非高峰期總等待時間：", wait_time_offpeak)
print("適應度：", -fitness(best_signal_offpeak, traffic_flow_offpeak, is_offpeak=True))  # 輸出非高峰期適應度

print("\n整體適應度：", wait_time_peak + wait_time_offpeak)  # 輸出非高峰期適應度

# 檢查是否過度優化
if wait_time_peak < 1500 or wait_time_offpeak < 500:
    print("\n警告：發現過度優化問題，等待時間過低，可能不符合實際情況！")


========== 高峰期結果 ==========
最佳信號燈綠燈時間配置： [85, 85, 85, 85]
高峰期總等待時間： 1788.75
適應度： 1788.75

========== 非高峰期結果 ==========
最佳信號燈綠燈時間配置： [34, 33, 33, 33]
非高峰期總等待時間： 693.4586466165413
適應度： 694.4586466165413

整體適應度： 2482.208646616541
